In [1]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import *
from reco_utils.recommender.deeprec.models.dkn_item2item import *
from reco_utils.recommender.deeprec.io.dkn_item2item_iterator import *
import time

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:

data_path = 'data_folder/my/DKN-training-folder'
yaml_file = './dkn.yaml' #os.path.join(data_path, r'../../../../../../dkn.yaml')
train_file = os.path.join(data_path, r'item2item_train_instances.txt')
valid_file = os.path.join(data_path, r'item2item_valid_instances.txt')
news_feature_file = os.path.join(data_path, r'../paper_feature.txt')
wordEmb_file = os.path.join(data_path, r'word_embedding.npy')
entityEmb_file = os.path.join(data_path, r'entity_embedding.npy')
contextEmb_file = os.path.join(data_path, r'context_embedding.npy')
infer_embedding_file = os.path.join(data_path, r'infer_embedding_item2item.txt')
news_feature_file = os.path.join(data_path,  r'../paper_feature.txt')


In [3]:
epoch = 10
hparams = prepare_hparams(yaml_file,
                          news_feature_file=news_feature_file,
                          wordEmb_file=wordEmb_file,
                          entityEmb_file=entityEmb_file,
                          contextEmb_file=contextEmb_file,
                          epochs=epoch,
                          is_clip_norm=True,
                          max_grad_norm=0.5,
                          his_size=20,
                          MODEL_DIR=os.path.join(data_path, 'save_models'),
                          learning_rate=0.0002,
                          embed_l2=0.0,
                          layer_l2=0.0,
                          batch_size=32,
                          use_entity=False,
                          use_context=False
                          )
hparams.neg_num=9
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

kg_file=None,user_clicks=None,FEATURE_COUNT=None,FIELD_COUNT=None,data_format=dkn,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=data_folder/my/DKN-training-folder/save_models,wordEmb_file=data_folder/my/DKN-training-folder/word_embedding.npy,entityEmb_file=data_folder/my/DKN-training-folder/entity_embedding.npy,contextEmb_file=data_folder/my/DKN-training-folder/context_embedding.npy,news_feature_file=data_folder/my/DKN-training-folder/../paper_feature.txt,user_history_file=None,use_entity=False,use_context=False,doc_size=15,history_size=

In [4]:
input_creator = DKNItem2itemTextIterator

In [9]:
model = DKNItem2Item(hparams, input_creator)

In [6]:
t01 = time.time()
print(model.run_eval(valid_file))
t02 = time.time()
print((t02 - t01) / 60)

{'group_auc': 0.84, 'mean_mrr': 0.6799, 'ndcg@2': 0.6291, 'ndcg@4': 0.7024, 'ndcg@6': 0.7331}
0.7933795015017192


In [7]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:51.840644601726616
eval info: group_auc:0.9486, mean_mrr:0.8653, ndcg@2:0.8575, ndcg@4:0.8875, ndcg@6:0.8937
at epoch 1 , train time: 53.5 eval time: 47.4
at epoch 2
train info: logloss loss:49.367171327154644
eval info: group_auc:0.95, mean_mrr:0.8747, ndcg@2:0.8691, ndcg@4:0.8934, ndcg@6:0.8998
at epoch 2 , train time: 49.8 eval time: 47.3
at epoch 3
train info: logloss loss:48.27005588421166
eval info: group_auc:0.9487, mean_mrr:0.8776, ndcg@2:0.8719, ndcg@4:0.8947, ndcg@6:0.9006
at epoch 3 , train time: 75.4 eval time: 52.0
at epoch 4
train info: logloss loss:47.53569007091205
eval info: group_auc:0.9482, mean_mrr:0.8807, ndcg@2:0.8747, ndcg@4:0.8963, ndcg@6:0.9021
at epoch 4 , train time: 75.6 eval time: 52.0
at epoch 5
train info: logloss loss:46.95931732150988
eval info: group_auc:0.9475, mean_mrr:0.8818, ndcg@2:0.8759, ndcg@4:0.8968, ndcg@6:0.9025
at epoch 5 , train time: 53.4 eval time: 48.6
at epoch 6
train info: logloss loss:46.50594117066

In [8]:
model.run_get_embedding(news_feature_file, infer_embedding_file)